# Laboratorium 3 - Nieparametryczne modele regresyjne

**Statystyczne Reguły Decyzyjne**

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#EDA" data-toc-modified-id="EDA-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>EDA</a></span></li><li><span><a href="#Splines" data-toc-modified-id="Splines-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Splines</a></span></li><li><span><a href="#Regresja-lokalna" data-toc-modified-id="Regresja-lokalna-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Regresja lokalna</a></span></li><li><span><a href="#Uogólnione-modele-addytywne-(GAM)" data-toc-modified-id="Uogólnione-modele-addytywne-(GAM)-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Uogólnione modele addytywne (GAM)</a></span></li></ul></div>

## EDA

In [ ]:
#!pip install csaps
#!pip install pygam

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt  
import seaborn as sns
import math
from scipy import stats
from scipy.interpolate import interp1d
import csaps
import copy
import statsmodels.api as sm
from pygam import LogisticGAM, l, s, f
from sklearn.datasets import load_breast_cancer
%matplotlib inline 

<br>

**Załadowanie i eksploracja danych**

In [ ]:
dataset = pd.read_fwf("https://raw.githubusercontent.com/KrainskiL/SLM_Z2021/master/Zajecia3/DATA4-12.txt", 
                     names = ["MORT", "INCC", "POV", "EDU1", "EDU2", "ALCC",
                     "TOBC", "HEXC", "PHYS", "URB", "AGED"])
dataset.head()

**Statystyki opisowe**

In [ ]:
dataset.describe()

<br>

**Macierze korelacji**

https://towardsdatascience.com/better-heatmaps-and-correlation-matrix-plots-in-python-41445d0f2bec

In [ ]:
corr = dataset.corr()
corr

In [ ]:
plt.rcParams['figure.figsize'] = [5,5]
ax = sns.heatmap(corr, square = True)

In [ ]:
#https://seaborn.pydata.org/generated/seaborn.diverging_palette.html
plt.rcParams['figure.figsize'] = [5,5]
ax = sns.heatmap(corr, square = True, cmap=sns.diverging_palette(20, 220, n=200))
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right');

In [ ]:
#Changing square size to show correlation strength
def heatmap(x, y, size):
    fig, ax = plt.subplots()
    
    # Mapping from column names to integer coordinates
    x_labels = [v for v in sorted(x.unique())]
    y_labels = [v for v in sorted(y.unique())]
    x_to_num = {p[1]:p[0] for p in enumerate(x_labels)} 
    y_to_num = {p[1]:p[0] for p in enumerate(y_labels)} 
    
    size_scale = 500
    ax.scatter(
        x=x.map(x_to_num), # Use mapping for x
        y=y.map(y_to_num), # Use mapping for y
        s=(size * size_scale).abs(), # Vector of square sizes, proportional to size parameter
        c=size* size_scale,
        marker='s' # Use square as scatterplot marker
    )
    
    # Show column labels on the axes
    ax.set_xticks([x_to_num[v] for v in x_labels])
    ax.set_xticklabels(x_labels, rotation=45, horizontalalignment='right')
    ax.set_yticks([y_to_num[v] for v in y_labels])
    ax.set_yticklabels(y_labels)

In [ ]:
corr = dataset.corr()
corr = pd.melt(corr.reset_index(), id_vars='index') # Unpivot the dataframe, so we can get pair of arrays for x and y
corr.columns = ['x', 'y', 'value']
heatmap(
    x=corr['x'],
    y=corr['y'],
    size=corr['value']
)

In [ ]:
#Caluclating p-value for each correlation value
corr = dataset.corr()
corr_new = copy.deepcopy(corr)
corr_p = copy.deepcopy(corr)
for i in np.arange(1,corr.shape[0]):
    for j in np.arange(1,corr.shape[1]):
        corr_p.iloc[i,j] = stats.stats.pearsonr( dataset[corr.index.values[i]], dataset[corr.columns[j]] )[1]

In [ ]:
indices = corr_p > 0.05
corr_new[indices] = 0
corr_new

In [ ]:
plt.rcParams['figure.figsize'] = [5,5]
ax = sns.heatmap(corr_new, square = True,
                vmin = -1, vmax = 1, center = 0,
                cmap=sns.diverging_palette(30, 150, n=200))

**Wykresy pudełkowe**

In [ ]:
fig, axes = plt.subplots(4,3,figsize=[15,10])
num_cols=dataset.select_dtypes(include=np.number).columns
for i, col in enumerate(num_cols):
    sns.boxplot(x=col,color=np.random.rand(3,), data=dataset, ax=axes[i//3,i%3],fliersize=1,width=0.6)
fig.delaxes(axes[3,2])
plt.tight_layout()

**Histogramy z estymacją funkcji gęstości**

In [ ]:
fig, axes = plt.subplots(4,3,figsize=[15,10])
num_cols=dataset.select_dtypes(include=np.number).columns
for i, col in enumerate(num_cols):
    sns.distplot(dataset[col],ax=axes[i//3,i%3])
fig.delaxes(axes[3,2])
plt.tight_layout()

<br>

**Wykresy rozrzutu**

In [ ]:
sns.set(style = "ticks")
sns.pairplot(dataset);

In [ ]:
fig, axes = plt.subplots(4,3,figsize=[15,10])
for i, col in enumerate(num_cols):
    c_ax=axes[i//3,i%3]
    sns.regplot(x=col,y='MORT',color=sns.color_palette("muted", n_colors=11)[i], 
                data=dataset, ax=c_ax, order=3)
    slope, intercept, *_ = stats.linregress(dataset[col],dataset['MORT'])
    line = slope*dataset[col]+intercept
    c_ax.plot(dataset[col], line, 'black', alpha=0.5)
fig.delaxes(axes[3,2])
plt.tight_layout()

## Splines

Smoothing spline $f(x)$ jest wyznaczany poprzez optymalizację poniższej formuły:
$$\sum_{i=1}^N(y_i-f(x_i))^2+\lambda \int_D(f^{''}(t))^2\textit{d}t \rightarrow min$$

gdzie $\lambda$ to parametr wygładzania. Optymalne rozwiązanie powyższej formuły jest zawsze ciagłym splinem złożonym z wielomianów 3 rzędu. 

In [ ]:
np.random.seed(1234)

x = np.linspace(-5., 5., 25)
y = np.exp(-(x/2.5)**2) + (np.random.rand(25) - 0.2) * 0.3
clrs = ['b','g','r','y']
for i in range(4):
    sp = csaps.CubicSmoothingSpline(x, y, smooth = i/3)
    xs = np.linspace(x[0], x[-1], 150)
    ys = sp(xs)
    plt.plot(x, y, 'o', xs, ys, clrs[i]+'-')
plt.show()

In [ ]:
fig, axes = plt.subplots(5,2,figsize=[15,10])
for i, col in enumerate(num_cols[1:]):
    c_ax=axes[i//2,i%2]
    sns.regplot(x=col,y='MORT',color=sns.color_palette("muted", n_colors=11)[i], 
                data=dataset, ax=c_ax, order=3)
    srted = dataset[[col,'MORT']].sort_values(by=col).drop_duplicates(subset=col)
    spline = csaps.CubicSmoothingSpline(srted[col],srted['MORT'], smooth=0.85)(srted[col])
    c_ax.plot(srted[col], spline, 'black', alpha=0.5)
plt.tight_layout()

## Regresja lokalna

Regresja lokalna (LOESS) działa następująco:

Dla danego punktu $x_0$ predykcja regresji LOESS drugiego stopnia (wartość domyślna dla funkcji `loess` w R) jest określona formułą:

$$\hat{f}(x_0)=\hat{\alpha}(x_0)+\hat{\beta}(x_0)x_0+\hat{\gamma}(x_0)x_0^2$$

Parametry $\alpha,\beta,\gamma$ obliczane są dla danego punktu $x_0$ bazując na zadaniu optymalizacyjnym:

$$\min_{\alpha(x_0),\beta(x_0),\gamma(x_0)} \sum_{i=1}^NK_{\lambda}(x_0,x_i)\left(\alpha(x_0)+\beta(x_0)x_i+\gamma(x_0)x_i^2-y_i\right)^2$$

gdzie $K_{\lambda}$ jest określone jako:

$$K_{\lambda}(x_0,x_i)=\phi\left(\frac{|x_0-x_i|}{\lambda}\right)$$

In [ ]:
housing = pd.read_fwf("http://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data",
                      names=['CRIM','ZN','INDUS','CHAS','NOX','RM','AGE','DIS','RAD','TAX','PTRATIO','LSTAT','MEDV'])

In [ ]:
plt.rcParams['figure.figsize'] = [10,5]
lstat_grid = np.linspace(np.asarray(min(housing['LSTAT'])), np.asarray(max(housing['LSTAT'])), 200)

# lowess will return our "smoothed" data with a y value for at every x-value
lowess = sm.nonparametric.lowess(housing['MEDV'], housing['LSTAT'], frac=.1)
lowess2 = sm.nonparametric.lowess(housing['MEDV'], housing['LSTAT'], frac=.9)

# unpack the lowess smoothed points to their values
lowess_x = list(zip(*lowess))[0]
lowess_y = list(zip(*lowess))[1]
f = interp1d(lowess_x, lowess_y, bounds_error=False)
ynew = f(lstat_grid)

lowess_x2 = list(zip(*lowess2))[0]
lowess_y2 = list(zip(*lowess2))[1]
f2 = interp1d(lowess_x2, lowess_y2, bounds_error=False)
ynew2 = f2(lstat_grid)

plt.plot(housing['LSTAT'], housing['MEDV'], 'o')
plt.plot(lstat_grid, ynew, '-', color = "red")
plt.plot(lstat_grid, ynew2, '-')
plt.title("Local regression")
plt.xlabel("Median value of the house ($1000s)")
plt.ylabel("% lower status of population")
plt.legend(['','0.1', '0.9'])
plt.show()

## Uogólnione modele addytywne (GAM)

https://pygam.readthedocs.io/en/latest/api/gam.html

**GLM**

$$g(E_Y(y|x))=\beta_0+\beta_1{}x_{1}+\ldots{}\beta_p{}x_{p}$$

$g()$ to funkcja linkująca (link). Na przykład, dla regresji logistycznej:

$$ln\left(\frac{E_Y(y|x)}{1-E_Y(y|x)}\right)=ln\left(\frac{P(y=1|x)}{1-P(y=1|x)}\right)=x^{T}\beta$$

**GAM**

$$g(E_Y(y|x))=\beta_0+f_1(x_{1})+f_2(x_{2})+\ldots+f_p(x_{p})$$

In [ ]:
cancer = load_breast_cancer()
cancer_df = pd.DataFrame(cancer.data, columns = cancer.feature_names)
y = pd.Series(cancer.target)

In [ ]:
X = cancer_df.iloc[:,0:3]
X

In [ ]:
# l - linear, s - spline
# others: f - factor, te - tensor
gam = LogisticGAM(l(0) + s(1) + s(2)).fit(X, y)
gam.summary()

In [ ]:
gam.accuracy(X, y)

In [ ]:
fig, axs = plt.subplots(1, X.shape[1])
titles = X.columns

for i, ax in enumerate(axs):
    XX = gam.generate_X_grid(term=i)
    pdep, confi = gam.partial_dependence(term=i, width=.95)

    ax.plot(XX[:, i], pdep)
    ax.plot(XX[:, i], confi, c='r', ls='--')
    ax.set_title(titles[i]);